# PLAYER LEAGUE UPDATE  

Este notebook atualiza as informações dos jogadores na LIGA e suas estatisticas sumarizadas pelo campeonato  

Obrigatorio passar LEAGUE_ID e SESON  
Atualizado:
<ul>
  <li>player</li>
  <li>player_statistics (sumarizado por campeonato)</li>
</ul>

> #TODO  
>  
> -> on conflict, atualizar alguns campos
> .


In [91]:
id_league = 71
season = 2021

In [92]:
import ipynb.fs.full.FootballApi as api
import ipynb.fs.full.PostgreDB as pg

In [93]:
#conn = pg.PostgreDB.connect()

In [94]:
#importando jogadores que jogaram liga especificada e suas statisticas
league_players = api.FootballApi(function=f"players?season={season}&league={id_league}").get_data()

result_per_page = league_players["results"]
paging = league_players["paging"]
current = paging["current"]
total = paging["total"]
page = current

In [95]:
#league_players = api.FootballApi(function=f"players?season={season}&team=134").get_data()
#print(league_players)
#print(f'Pagina {current}/{total} - [{result_per_page} rows]')

In [ ]:
import time

#players = league_players['response']

# função para descobrir o id country
def get_id_country(country:str) -> int:

    conn = pg.PostgreDB.connect()

    sql = f" SELECT id FROM country WHERE name = '{country}' "
    cur = pg.PostgreDB.run_query(conn, sql)
    result = cur.fetchone()

    if result: 
        id_country = result[0]
    else:
        id_country = 0

    pg.PostgreDB.commit(conn)
    pg.PostgreDB.close(conn)

    return int(id_country)

# funcao para limpar ints que vem com string (Ex: 191cm -> 191)
def remove_string(my_string:str) -> int:
    only_int = []
    for i in my_string:
        if i.isdigit():
            only_int.append(i)

    result = ''.join(only_int)
    return int(result)

#funcao para salvar no banco os dados
def insert_db(table: str, data: dict) -> None:

    conn = pg.PostgreDB.connect()
    
    columns = ", ".join([list(data.keys()) for x in data][0])
    s_len = ", ".join(["%s"] * len([list(data.keys()) for x in data][0]))
    sql = f"""INSERT INTO {table} ({columns})
                VALUES ({s_len}) ON CONFLICT 
                DO NOTHING;"""

    cur = pg.PostgreDB.run_query_with_args(conn, sql, data)
    pg.PostgreDB.commit(conn)
    pg.PostgreDB.close(conn)


while page <= total:
    
    league_players = api.FootballApi(function=f"players?season={season}&league={id_league}&page={page}").get_data()
    players = league_players['response']
    print(f'Pagina {page}/{total} - [{result_per_page} players]')
    #print("Pagina: "+ str(page))
    page += 1
    print("Inseriondo dados...")

    # dicionarios que vão conter os dados exatos e limpos para serem inseridos no banco
    player_data = {}
    player_statistic_data = {}

    # for principal para salvar dados
    for player in players:
        #print(player)
        for (k, v) in player.items():
            if k == 'player':
                player_data['id'] = v['id']
                player_data['name'] = v['name']
                player_data['firstname'] = v['firstname']
                player_data['lastname'] = v['lastname']
                player_data['birth_date'] = v['birth']['date']
                player_data['id_country'] = get_id_country(v['birth']['country'])
                player_data['id_nationality'] = get_id_country(v['nationality'])
                player_data['height'] = remove_string(v['height']) if v['height'] is not None else 0
                player_data['weight'] = remove_string(v['weight']) if v['weight'] is not None else 0
                player_data['injured'] = v['injured']
                player_data['photo'] = v['photo']
                insert_db(k, player_data)
                
            elif k == 'statistics':
                player_statistic_data['id_player'] = player_data['id']
                player_statistic_data['id_team'] = v[0]['team']['id']
                player_statistic_data['id_league'] = v[0]['league']['id']
                player_statistic_data['season'] = v[0]['league']['season']
                player_statistic_data['game_appearences'] = v[0]['games']['appearences']
                player_statistic_data['game_lineups'] = v[0]['games']['lineups']
                player_statistic_data['game_minutes'] = v[0]['games']['minutes']
                player_statistic_data['game_number'] = v[0]['games']['number']
                player_statistic_data['game_position'] = v[0]['games']['position']
                player_statistic_data['game_rating'] = v[0]['games']['rating']
                player_statistic_data['game_captain'] = v[0]['games']['captain']
                player_statistic_data['substitutes_in'] = v[0]['substitutes']['in']
                player_statistic_data['substitutes_out'] = v[0]['substitutes']['out']
                player_statistic_data['substitutes_bench'] = v[0]['substitutes']['bench']
                player_statistic_data['shots_total'] = v[0]['shots']['total']
                player_statistic_data['shots_on'] = v[0]['shots']['on']
                player_statistic_data['goals_total'] = v[0]['goals']['total']
                player_statistic_data['goals_conceded'] = v[0]['goals']['conceded']
                player_statistic_data['goals_assists'] = v[0]['goals']['assists']
                player_statistic_data['goals_saves'] = v[0]['goals']['saves']
                player_statistic_data['passes_total'] = v[0]['passes']['total']
                player_statistic_data['passes_key'] = v[0]['passes']['key']
                player_statistic_data['passes_accuracy'] = v[0]['passes']['accuracy']
                player_statistic_data['tackles_total'] = v[0]['tackles']['total']
                player_statistic_data['tackles_blocks'] = v[0]['tackles']['blocks']
                player_statistic_data['tackles_interceptions'] = v[0]['tackles']['interceptions']
                player_statistic_data['duels_total'] = v[0]['duels']['total']
                player_statistic_data['duels_won'] = v[0]['duels']['won']
                player_statistic_data['dribbles_attempts'] = v[0]['dribbles']['attempts']
                player_statistic_data['dribbles_success'] = v[0]['dribbles']['success']
                player_statistic_data['dribbles_past'] = v[0]['dribbles']['past']
                player_statistic_data['fouls_drawn'] = v[0]['fouls']['drawn']
                player_statistic_data['fouls_committed'] = v[0]['fouls']['committed']
                player_statistic_data['cards_yellow'] = v[0]['cards']['yellow']
                player_statistic_data['cards_yellowred'] = v[0]['cards']['yellowred']
                player_statistic_data['cards_red'] = v[0]['cards']['red']
                player_statistic_data['penalty_won'] = v[0]['penalty']['won']
                player_statistic_data['penalty_commited'] = v[0]['penalty']['commited']
                player_statistic_data['penalty_scored'] = v[0]['penalty']['scored']
                player_statistic_data['penalty_missed'] = v[0]['penalty']['missed']
                player_statistic_data['penalty_saved'] = v[0]['penalty']['saved']
                
                table_name = "player_league_" + k
                insert_db(table_name, player_statistic_data)
                #print(player_statistic_data)


    print("Sleep de 6s por conta da limitacao da API no plano BASICO...")
    time.sleep(6)
